In [2]:
# Necessary Packages to be installed before running the code
# pip install google_spreadsheet
# pip install google-auth-oauthlib
# pip install pandas

### Business Metadata

In [3]:
# Connecting to the google sheet for Business Metadata
import pandas as pd
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow,Flow
from google.auth.transport.requests import Request
import os
import pickle

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# Here enter the ID of your Google Sheet
SAMPLE_SPREADSHEET_ID_input = '18YdO9N_KmMV57-l_2utxQ-i5AH2A66XCEnDV1Lyb_zs'
SAMPLE_RANGE_NAME = 'Final!A1:D'

def main():
    global values_input, service
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'class_credentials.json', SCOPES) # here enter the name of your downloaded JSON file
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result_input = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID_input,
                                range=SAMPLE_RANGE_NAME).execute()
    values_input = result_input.get('values', [])

    if not values_input and not values_expansion:
        print('No data found.')

main()

# Creates a dataframe with all the records
df=pd.DataFrame(values_input[1:], columns=values_input[0])

In [4]:
# Displaying the dataframe
df

,DomainName,DBName,BusinessTerm_Name,Term_Description
0,Team1,Weather Database,Event ID,Number assigned per Event occured
1,Team1,Weather Database,Event Type,"Type of the event like rain , snow"
2,Team1,Weather Database,Event Severity,Metrics do describe how severe the event
3,Team1,Weather Database,Event Start Time,Time stamp for date when event has occured
4,Team1,Weather Database,Event End Time,Time stamp for date when event has completed
...,...,...,...,...
161,Team3,projectDB,lat,latitude of the location
162,Team3,projectDB,long,longitude of the location
163,Team3,projectDB,posting_date,date when the craiglist post was made
164,Team3,projectDB,vehicle_id,"unique identifier for node ""vehicle"""


In [5]:
# Connection for using MSSQL Server
import pyodbc

connection = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER=PV-DESKTOP\SQLSERVERDEV19;DATABASE=Metadata_class_Project;UID=pv2612;PWD=classproject')

In [6]:
# Using cursor to store the result of queries
cursor = connection.cursor()

In [7]:
# Query for Schema Information
cursor.execute("SELECT * FROM INFORMATION_SCHEMA.TABLES")

In [8]:
# Displaying all the tables in the database
for i in cursor:
    print(i)

('Metadata_class_project', 'dbo', 'Domain', 'BASE TABLE')
('Metadata_class_project', 'dbo', 'BusinessTerm', 'BASE TABLE')
('Metadata_class_project', 'dbo', 'Nodes', 'BASE TABLE')
('Metadata_class_project', 'dbo', 'Property', 'BASE TABLE')
('Metadata_class_project', 'dbo', 'Relationship', 'BASE TABLE')
('Metadata_class_project', 'dbo', 'DomBusBrdge', 'BASE TABLE')
('Metadata_class_project', 'dbo', 'BusPropBridge', 'BASE TABLE')


### Domain Table Script

In [28]:
# Creating a dataframe to load domain tabel
df_dom = df[['DBName','DomainName']]
domainList = df_dom.drop_duplicates().reset_index(drop=True)
domainList

,DBName,DomainName
0,Weather Database,Team1
1,HotelReview,Team5
2,Commodity,Team6
3,BuildingPermits,Team7
4,USPermVisas,Team8
5,COVID19,Team2
6,Consumer_Complaints,Team4
7,projectDB,Team3


In [29]:
# Truncate Query for Domain Table
cursor.execute("Truncate table Domain")

In [30]:
# Insert Query for Domain Table
query = "Insert into Domain (DBName, DomainName) values(?,?)";
for row in domainList.iterrows():
    val = (row[1][0], row[1][1]);
    cursor.execute(query,val);
connection.commit()

In [31]:
cursor.execute("Select * from Domain");

In [32]:
# Creating Dictionary for Domain Table
domain = {};
for i in cursor:
    print(i[0],i[1]);
    domain[i[0]]=i[1];

BuildingPermits Team7
Commodity Team6
Consumer_Complaints Team4
COVID19 Team2
HotelReview Team5
projectDB Team3
USPermVisas Team8
Weather Database Team1


In [33]:
domain

{'BuildingPermits': 'Team7',
 'Commodity': 'Team6',
 'Consumer_Complaints': 'Team4',
 'COVID19': 'Team2',
 'HotelReview': 'Team5',
 'projectDB': 'Team3',
 'USPermVisas': 'Team8',
 'Weather Database': 'Team1'}

### Creating a Helper Function

In [14]:
# # Helper function to get the key of a dictionary from given value
def get_key(val,dicti):
    for key, value in dicti.items():
         if val == value:
             return key

    return "key doesn't exist"

In [15]:
get_key('Team5',domain)

'HotelReview'

### BusinessTerms Table Script

In [16]:
# Insert Query for BusinessTerm Table
df_bus = df[['BusinessTerm_Name','Term_Description']]
BusList = df_bus.drop_duplicates().reset_index(drop=True)
BusList

,BusinessTerm_Name,Term_Description
0,Event ID,Number assigned per Event occured
1,Event Type,"Type of the event like rain , snow"
2,Event Severity,Metrics do describe how severe the event
3,Event Start Time,Time stamp for date when event has occured
4,Event End Time,Time stamp for date when event has completed
...,...,...
161,lat,latitude of the location
162,long,longitude of the location
163,posting_date,date when the craiglist post was made
164,vehicle_id,"unique identifier for node ""vehicle"""


In [17]:
# Truncate Query for Domain Table
cursor.execute("Truncate table BusinessTerm")

In [18]:
# Insert Query for BusinessTerm Table
query = "Insert into BusinessTerm (BusType, BusinessDesc) values(?,?)";
for row in BusList.iterrows():
    val = (row[1][0], row[1][1]);
    cursor.execute(query,val);
connection.commit()

In [19]:
# Displaying the values in the BusinessTerm Table
cursor.execute("Select * from BusinessTerm");

In [20]:
# Creating Dictionary for BusinessTerm Table
busterms = {}; #key:bustermId; val:busTerm
for i in cursor:
    print(i[0],i[1],i[2]);
    busterms[i[0]]=(i[2]);

100 Number assigned per Event occured Event ID
101 Type of the event like rain , snow Event Type
102 Metrics do describe how severe the event  Event Severity
103 Time stamp for date when event has occured Event Start Time
104 Time stamp for date when event has completed Event End Time
105 The US-Based time zone based on the location of the event Time-Zone
106 The airport station that a weather event is reported from. Airport Code
107 The latitude in GPS coordinate of airport-based weather station. Latitude Location
108 The longitude in GPS coordinate of airport-based weather station. Longitude Location
109 The cityof airport-based weather station. City
110 The county of airport-based weather station. County
111 The state of airport-based weather station. State
112 The zip code of airport-based weather station. Zip Code
113 The duration in minutes  Duration
114 Address of hotel Hotel_Address
115 There are also some guests who just made a scoring on the service rather than a review. This

246 manufacturing company of the vehicle manufacturer
247 model name model
248 condition of the vehicle condition
249 number of cylinders cylinders
250 type of fuel the vehicle runs on fuel
251 kilometers the vehicle has been driven odometer
252 condition of vehicle in terms of quality title_status
253 gear transmission used by the vehicle transmission
254 vehicle identity number VIN
255 type of drive the vehicle possess drive
256 size of the vehicle size
257 category of the vehicle model type
258 paint color on the vehicle paint_color
259 link to vehicle's image image_url
260 state where the craiglist post was made state
261 latitude of the location lat
262 longitude of the location long
263 date when the craiglist post was made posting_date
264 unique identifier for node "vehicle" vehicle_id
265 unique identifier for node "condition" condition_id


In [21]:
get_key('Event ID',busterms)

100

### DomainTermsRelationship Table Script

In [22]:
# Truncate Query for DomBusBrdge Table
cursor.execute("Truncate table DomBusBrdge")

In [23]:
# Insert Query for DomBusBrdge Table
query = "Insert into DomBusBrdge (DBName, BusinessId) values(?,?)";
for row in df.iterrows():
    domainName = row[1][0]
    busTermName = row[1][2]
    domId = get_key(domainName,domain)
    busId = get_key(busTermName,busterms)
    val = (domId,busId)
    cursor.execute(query,val)
connection.commit()

In [24]:
# Displaying the values in the DomBusBrdge Table
cursor.execute("Select * from DomBusBrdge");

In [25]:
# Values in Domain BusinessTerm Bridge Table 
for i in cursor:
    print(i)

('BuildingPermits', 148)
('BuildingPermits', 149)
('BuildingPermits', 150)
('BuildingPermits', 151)
('BuildingPermits', 152)
('BuildingPermits', 153)
('BuildingPermits', 154)
('BuildingPermits', 155)
('BuildingPermits', 156)
('BuildingPermits', 157)
('BuildingPermits', 158)
('BuildingPermits', 159)
('BuildingPermits', 160)
('BuildingPermits', 161)
('BuildingPermits', 162)
('BuildingPermits', 163)
('BuildingPermits', 164)
('BuildingPermits', 165)
('BuildingPermits', 166)
('BuildingPermits', 167)
('BuildingPermits', 168)
('BuildingPermits', 169)
('BuildingPermits', 170)
('BuildingPermits', 171)
('BuildingPermits', 172)
('BuildingPermits', 173)
('BuildingPermits', 174)
('BuildingPermits', 175)
('BuildingPermits', 176)
('BuildingPermits', 177)
('BuildingPermits', 178)
('BuildingPermits', 179)
('BuildingPermits', 180)
('BuildingPermits', 181)
('BuildingPermits', 182)
('BuildingPermits', 183)
('BuildingPermits', 184)
('BuildingPermits', 185)
('BuildingPermits', 186)
('BuildingPermits', 187)


### Technical Metadata

In [26]:
# Packages needed to read Google sheet
import pandas as pd
import pygsheets
import py2neo
from pandas import DataFrame
from py2neo import Graph

In [27]:
# Openning Google Sheet for Technical Metadata
gc = pygsheets.authorize(service_file='service_account_1.json')
sh = gc.open('csye7250-metadata')

In [28]:
# Create an initial dataframe with google sheet1
df = sh[1]
df = DataFrame(df)
df = df.iloc[:,0:10]
df.columns = df.iloc[0]
df = df[1:]

# Append all the remaining sheets data to df dataframe
for i in range (2,9):
    df1 = sh[i]
    df1 = DataFrame(df1)
    df1 = df1.iloc[:,0:10]
    df1.columns = df1.iloc[0]
    df1 = df1[1:]
    if df1.empty == False:
        df = df.append(df1, ignore_index = True)

In [29]:
#df has data from all teams converted into a dataframe
df

,counts,label,property,type,isIndexed,uniqueConstraint,existenceConstraint,team,dbName,relationships
0,6274206,EVENT,StartTime,DATE_TIME,FALSE,FALSE,FALSE,1,Weather Database,"COUNTY_OF,IS_OF,WITH,SENSOR_AT,CITY_OF,STATE_O..."
1,6274206,EVENT,TimeZone,STRING,FALSE,FALSE,FALSE,1,Weather Database,"COUNTY_OF,IS_OF,WITH,SENSOR_AT,CITY_OF,STATE_O..."
2,6274206,EVENT,Duration,INTEGER,FALSE,FALSE,FALSE,1,Weather Database,"COUNTY_OF,IS_OF,WITH,SENSOR_AT,CITY_OF,STATE_O..."
3,6274206,EVENT,Id,STRING,TRUE,TRUE,FALSE,1,Weather Database,"COUNTY_OF,IS_OF,WITH,SENSOR_AT,CITY_OF,STATE_O..."
4,6274206,EVENT,EndTime,DATE_TIME,FALSE,FALSE,FALSE,1,Weather Database,"COUNTY_OF,IS_OF,WITH,SENSOR_AT,CITY_OF,STATE_O..."
...,...,...,...,...,...,...,...,...,...,...
1130,6833,City,name,STRING,TRUE,TRUE,FALSE,8,USPermVisas,"Location,Of"
1131,58,State,name,STRING,TRUE,TRUE,FALSE,8,USPermVisas,"Location,Of"
1132,10,Wage,range,STRING,TRUE,TRUE,FALSE,8,USPermVisas,Has
1133,10,Wage,level,STRING,FALSE,FALSE,FALSE,8,USPermVisas,Has


### Nodes Table Script

In [30]:
# Get only the needed columns from complete data for nodes table and take unique combinations of them
df_node = df[['counts','label','dbName']]
nodeList = df_node.drop_duplicates().reset_index(drop=True)
nodeList

,counts,label,dbName
0,6274206,EVENT,Weather Database
1,7,TYPE,Weather Database
2,6,SEVERITY,Weather Database
3,2071,AIRPORT,Weather Database
4,1715,CITY,Weather Database
5,48,STATE,Weather Database
6,2021,ZIPCODE,Weather Database
7,1100,COUNTY,Weather Database
8,224,Country,COVID19
9,966,Province,COVID19


In [31]:
# Truncate Query for Nodes Table
cursor.execute("Truncate table Nodes")

In [32]:
# Query for Nodes table
query = "Insert into Nodes (Label, Counts, DBName) values(?,?,?)";
for row in nodeList.iterrows():
    val = (row[1][1], row[1][0], row[1][2]);
    cursor.execute(query,val);
connection.commit()

In [33]:
# Displaying values in the Nodes table
cursor.execute("select * from Nodes");

In [34]:
# Putting Node data in a dictionary (nodeId : Label) as key:val pairs
nodeDict = {};
for i in cursor:
    nodeDict[i[0]]=(i[1]);

In [35]:
nodeDict

{1: 'EVENT',
 2: 'TYPE',
 3: 'SEVERITY',
 4: 'AIRPORT',
 5: 'CITY',
 6: 'STATE',
 7: 'ZIPCODE',
 8: 'COUNTY',
 9: 'Country',
 10: 'Province',
 11: 'Date',
 12: 'InfectionStatus',
 13: 'Complaint',
 14: 'Company',
 15: 'Response',
 16: 'Product',
 17: 'SubProduct',
 18: 'State',
 19: 'Zipcode',
 20: 'Issue',
 21: 'SubIssue',
 22: 'Region',
 23: 'Listing',
 24: 'Vehicle',
 25: 'Manufacturer',
 26: 'Model',
 27: 'V_Type',
 28: 'V_Condition',
 29: '',
 30: 'Hotel',
 31: 'Reviewer',
 32: 'Country',
 33: 'Date',
 34: 'Trip',
 35: 'Commodity',
 36: 'Category',
 37: 'Country',
 38: 'Flow',
 39: 'Year',
 40: 'PermitRecord',
 41: 'PermitType',
 42: 'BuildingProperty',
 43: 'CurrentStatus',
 44: 'PlanSet',
 45: 'SitePermit',
 46: 'FirePermit',
 47: 'Retrofit',
 48: 'StrucNotif',
 49: 'TidfComp',
 50: 'Case',
 51: 'Country',
 52: 'Decision',
 53: 'Type',
 54: 'SOC',
 55: 'Employer',
 56: 'Job',
 57: 'City',
 58: 'State',
 59: 'Wage',
 60: 'Sector'}

### Creating a helper function

In [36]:
# # Helper function to get the key of a dictionary from given value
# def get_key(val,dicti):
#     for key, value in dicti.items():
#          if val == value:
#              return key
 
#     return "key doesn't exist"

### Property Table Script

In [37]:
# Get only the needed columns from complete data for property table and take unique combinations of them
df_prop = df[['property','uniqueConstraint','existenceConstraint','label']]
propList = df_prop.drop_duplicates().reset_index(drop=True)
propList

,property,uniqueConstraint,existenceConstraint,label
0,StartTime,FALSE,FALSE,EVENT
1,TimeZone,FALSE,FALSE,EVENT
2,Duration,FALSE,FALSE,EVENT
3,Id,TRUE,FALSE,EVENT
4,EndTime,FALSE,FALSE,EVENT
...,...,...,...,...
158,name,TRUE,FALSE,City
159,name,TRUE,FALSE,State
160,range,TRUE,FALSE,Wage
161,level,FALSE,FALSE,Wage


In [38]:
# Truncate Query for Property Table
cursor.execute("Truncate table Property")

In [39]:
# Query for Property table
query = "Insert into Property (TechTerm, UniqueConstraints, ExistingConstraints, NodeId) values(?,?,?,?)";
for row in propList.iterrows():
    val = (row[1][0], row[1][1], row[1][2], get_key(row[1][3], nodeDict));
    cursor.execute(query,val);
connection.commit()

In [40]:
# Displaying the values in Property Table
cursor.execute("select * from Property");

In [41]:
# put property data in a dictionary (PropertyId : TechTerm) as key:val pairs
propDict = {};
for i in cursor:
    propDict[i[0]]=(i[1]);

In [42]:
propDict

{10: 'StartTime',
 11: 'TimeZone',
 12: 'Duration',
 13: 'Id',
 14: 'EndTime',
 15: 'Name',
 16: 'Name',
 17: 'AirportCode',
 18: 'LocationLng',
 19: 'LocationLat',
 20: 'Name',
 21: 'Name',
 22: 'ZipCode',
 23: 'Name',
 24: 'CountryId',
 25: 'CountryName',
 26: 'ProvinceName',
 27: 'ProvinceId',
 28: 'ObservationMonth',
 29: 'ObservationYear',
 30: 'ObservationDate',
 31: 'UpdateTimeYear',
 32: 'UpdateTimeMonth',
 33: 'UpdateTime',
 34: 'Recovered',
 35: 'EnteryId',
 36: 'Deaths',
 37: 'Confirmed',
 38: 'ComplaintId',
 39: 'DateReceived',
 40: 'name',
 41: 'companyResponse',
 42: 'ProductName',
 43: 'SubProductName',
 44: 'StateName',
 45: 'ZipId',
 46: 'IssueName',
 47: 'SubIssueName',
 48: 'regionName',
 49: 'region_url',
 50: 'state',
 51: 'listing_id',
 52: 'URL',
 53: 'posting_date',
 54: 'price',
 55: 'image_url',
 56: 'lat',
 57: 'long',
 58: 'vehicle_id',
 59: 'paint_color',
 60: 'VIN',
 61: 'manufacturerName',
 62: 'modelName',
 63: 'year',
 64: 'fuel',
 65: 'drive',
 66: 'tr

### Relationship Table Script

In [43]:
# Creating a dataframe to load Relationship tabel
df_rel = df[['label','relationships']]
relList = df_rel.drop_duplicates().reset_index(drop=True)
relList

,label,relationships
0,EVENT,"COUNTY_OF,IS_OF,WITH,SENSOR_AT,CITY_OF,STATE_O..."
1,TYPE,IS_OF
2,SEVERITY,WITH
3,AIRPORT,"SENSOR_AT,AT"
4,CITY,"CITY_OF,IN,AT"
...,...,...
56,Job,"In,For"
57,City,"Location,Of"
58,State,"Location,Of"
59,Wage,Has


In [44]:
# Truncate Query for Property Table
cursor.execute("Truncate table Relationship")

In [45]:
relList.relationships = relList.relationships.str.split(',')
relList = relList.explode('relationships').reset_index(drop=True)
relList = relList.drop_duplicates().reset_index(drop=True)
relList

,label,relationships
0,EVENT,COUNTY_OF
1,EVENT,IS_OF
2,EVENT,WITH
3,EVENT,SENSOR_AT
4,EVENT,CITY_OF
...,...,...
124,City,Of
125,State,Location
126,State,Of
127,Wage,Has


In [46]:
# Query for Relationship table
query = "Insert into Relationship (RelDesc, NodeId) values(?,?)";
for row in relList.iterrows():
    val = (row[1][1], get_key(row[1][0], nodeDict));
    cursor.execute(query,val);
connection.commit()

In [47]:
# Displaying values in Relationship Tables
cursor.execute("select * from Relationship");

In [48]:
for i in cursor:
    print(i)

(200, 'COUNTY_OF', 1)
(201, 'IS_OF', 1)
(202, 'WITH', 1)
(203, 'SENSOR_AT', 1)
(204, 'CITY_OF', 1)
(205, 'STATE_OF', 1)
(206, 'ZIPCODE_OF', 1)
(207, 'IS_OF', 2)
(208, 'WITH', 3)
(209, 'SENSOR_AT', 4)
(210, 'AT', 4)
(211, 'CITY_OF', 5)
(212, 'IN', 5)
(213, 'AT', 5)
(214, 'STATE_OF', 6)
(215, 'IN', 6)
(216, 'HAS', 6)
(217, 'ZIPCODE_OF', 7)
(218, 'HAS', 7)
(219, 'LOCATION', 7)
(220, 'COUNTY_OF', 8)
(221, 'LOCATION', 8)
(222, 'HAS', 9)
(223, 'BE_OBSERVED', 9)
(224, 'HAS', 10)
(225, 'BE_OBSERVED', 10)
(226, 'DEFINED', 10)
(227, 'BE_OBSERVED', 11)
(228, 'DEFINED', 11)
(229, 'DEFINED', 12)
(230, 'WITH', 13)
(231, 'AGAINST', 13)
(232, 'ORIGINATED_FROM', 13)
(233, 'ABOUT', 13)
(234, 'AGAINST', 14)
(235, 'FROM', 14)
(236, 'FROM', 15)
(237, 'ABOUT', 16)
(238, 'CONTAINS', 16)
(239, 'ABOUT', 17)
(240, 'CONTAINS', 17)
(241, 'ORIGINATED_FROM', 18)
(242, 'FALLS_UNDER', 18)
(243, 'FALLS_UNDER', 19)
(244, 'ORIGINATED_FROM', 19)
(245, 'WITH', 20)
(246, 'IN_CATEGORY', 20)
(247, 'WITH', 21)
(248, 'IN_CATEG